## Prediction using CNN and RNN

In [1]:
from keras.models import Sequential
from keras.layers import Convolution1D, Flatten, Dense, Activation, Dropout, LSTM, MaxPooling1D, BatchNormalization
from keras.optimizers import SGD, Adam, RMSprop
from FileReader import *
from Evaluation import *
from GenBatch import Getter
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import numpy as np
import pickle
import Constant as C
import matplotlib.pyplot as plt
import theano.tensor as T
sgd = SGD(lr=1e-3, momentum=0.9, decay=1e-6, nesterov=True)


def CNNconstruction():
    model = Sequential()
    model.add(Convolution1D(16, 3, input_shape=(22, C.nb_data-1), init="constant"))
    # model.add(BatchNormalization())
    model.add(Activation("relu"))

    model.add(Convolution1D(32, 3, init="constant"))
    # model.add(BatchNormalization())
    model.add(Activation("relu"))

    model.add(Convolution1D(64, 3, init="constant"))
    # model.add(BatchNormalization())
    model.add(Activation("relu"))

    model.add(MaxPooling1D(2))

    model.add(Flatten())
    # model.add(Dense(128, init="he_normal"))
    # model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, init="constant"))
#     model.add(Activation("tanh"))
    model.compile(optimizer=Adam(), loss='mse')
    return model


def RNNconstruction():
    model = Sequential()
    model.add(LSTM(32, input_shape=(C.nb_data-1, 22), return_sequences=False, consume_less='gpu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.compile(optimizer=Adam(), loss='mse')
    return model


def Complex(dataset):
    part1 = np.diff(dataset[:, :, :11], axis=1)
    part2 = dataset[:, :-1, 11:-1] / dataset[:, 1:, 11:-1]
    part3 = dataset[:, :-1, -1] - dataset[:, 1:, -1]
    part3 = np.expand_dims(part3, axis=2)
    part3 = np.tanh(part3)
    return np.concatenate((part1, part2, part3), axis=2)


def Diff(dataset):
    return np.diff(dataset, axis=1)


def Flat(dataset):
    return dataset.reshape(dataset.shape[0], np.prod(dataset.shape[1:]))


def Descale(dataset):
    dataset[:, :, :11] -= 100
    return dataset


def Zscore(dataset):
    cache = {}
    mean = dataset.mean(axis=0)
    dataset -= mean
    std = np.std(dataset, axis=0)
    dataset /= std
    cache["mean"] = mean
    cache["std"] = std
    return dataset, cache

########################################################################################################################################

model = CNNconstruction()
config = model.get_config()
conf_file = open("/home/wjchen/Documents/rhyang/ANN_configuration.txt", "wb")
pickle.dump(config, conf_file)
conf_file.close()

# callbacks
ES = EarlyStopping(monitor='val_loss', patience=1, verbose=1)
MC = ModelCheckpoint("/home/wjchen/Documents/rhyang/model.h5", monitor='val_loss', save_best_only=True)
########################################################################################################################################
X_training, y_training = Getter("/home/wjchen/Documents/rhyang/data.h5", 200, constraint=None)
X_training = Diff(X_training)
X_training = X_training.transpose(0,2,1)

X_eval, y_eval = Getter("/home/wjchen/Documents/rhyang/data_val.h5", 100, constraint=None)
X_eval = Diff(X_eval)
X_eval = X_eval.transpose(0,2,1)

R2 = 0

Using TensorFlow backend.


ImportError: No module named 'tensorflow'

In [2]:
for i in range(C.nb_epoch):
    print("EPOCH: " + str(i + 1) + "/" + str(C.nb_epoch))
    model.fit(X_training, y_training, nb_epoch=1, batch_size=C.batch_size,
              verbose=2, callbacks=[])

    evaluation_result = model.predict(X_eval)
    R2_temp = R_square(y_eval, evaluation_result.flatten())
    print("Out-sample R2: " + str(R2_temp))

    if R2_temp > R2:
        model.save_weights("/home/rhyang/Documents/best_weights.h5", overwrite=True)
        R2 = R2_temp
    model.save_weights("/home/rhyang/Documents/weights.h5", overwrite=True)

EPOCH: 1/100
Epoch 1/1
21s - loss: 7.2115e-05
18.9197893111 18.8869
Out-sample R2: -0.00173915933141
EPOCH: 2/100
Epoch 1/1
21s - loss: 7.2161e-05
18.8952816664 18.8869
Out-sample R2: -0.000441562042695
EPOCH: 3/100
Epoch 1/1
21s - loss: 7.2161e-05
18.896817224 18.8869
Out-sample R2: -0.000522864644344
EPOCH: 4/100
Epoch 1/1
23s - loss: 7.2182e-05
18.887109045 18.8869
Out-sample R2: -8.84924658084e-06
EPOCH: 5/100
Epoch 1/1
22s - loss: 7.2154e-05
18.8913167307 18.8869
Out-sample R2: -0.000231632043887
EPOCH: 6/100
Epoch 1/1
22s - loss: 7.2144e-05
19.247940878 18.8869
Out-sample R2: -0.0191136802336
EPOCH: 7/100
Epoch 1/1
22s - loss: 7.2209e-05
18.9495761282 18.8869
Out-sample R2: -0.00331627103474
EPOCH: 8/100
Epoch 1/1
22s - loss: 7.2141e-05
18.9197946439 18.8869
Out-sample R2: -0.00173944168715
EPOCH: 9/100
Epoch 1/1
22s - loss: 7.2192e-05


KeyboardInterrupt: 

In [ ]:
plt.figure(0)
plt.hist(evaluation_result.flatten(), bins=10000, histtype="step")
plt.figure(1)
plt.hist(y_eval, bins=10000, histtype="step")
plt.show()